In [5]:
from machine_learning_hep.listfiles import list_files_dir_lev2, list_files_lev2
from machine_learning_hep.doskimming import skimall

In [6]:
import yaml
with open("default_complete.yaml", 'r') as run_config: 
    data_config = yaml.load(run_config) 
with open("data/database_ml_parameters.yml", 'r') as param_config: 
    data_param = yaml.load(param_config) 
with open("data/config_model_parameters.yml", 'r') as mod_config: 
    data_model = yaml.load(mod_config) 
mcordata = "data"
indexp = 0
case = data_config["case"]
param_case = data_param[case]


def list_create_dir(inputdir, outputdir, namea, nameb, namec, nameaout, namebout, namecout, maxfiles):
    lista, listaout = list_files_dir_lev2(inputdir, outputdir, namea, nameaout)
    listb, listbout = list_files_dir_lev2(inputdir, outputdir, nameb, namebout)
    listc, listcout = list_files_dir_lev2(inputdir, outputdir, namec, namecout)

    if maxfiles is not -1:
        lista = lista[:maxfiles]
        listb = listb[:maxfiles]
        listc = listc[:maxfiles]
        listaout = listaout[:maxfiles]
        listbout = listbout[:maxfiles]
        listcout = listcout[:maxfiles]

        return lista, listb, listc, listaout, listbout, listcout

def createchunks(listin, listout, maxperchunk):
    chunks = [listin[x:x+maxperchunk]  for x in range(0, len(listin), maxperchunk)]
    chunksout = [listout[x:x+maxperchunk] for x in range(0, len(listout), maxperchunk)]
    return chunks, chunksout


In [7]:
namefile_reco = data_param[case]["files_names"]["namefile_reco"]
namefile_evt = data_param[case]["files_names"]["namefile_evt"]
namefile_gen = data_param[case]["files_names"]["namefile_gen"]
namefile_reco_skim = data_param[case]["files_names"]["namefile_reco_skim"]
namefile_evt_skim = data_param[case]["files_names"]["namefile_evt_skim"]
namefile_gen_skim = data_param[case]["files_names"]["namefile_gen_skim"]
var_evt_match = data_param[case]["variables"]["var_evt_match"]
skimming_sel = data_param[case]["skimming2_sel"]
skimming_sel_gen = data_param[case]["skimming2_sel_gen"]
skimming_sel_evt = data_param[case]["skimming2_sel_evt"]
presel_reco = data_param[case]["presel_reco"]
sel_cent = data_param[case]["sel_cent"]
skimming2_dotrackpid = data_param[case]["skimming2_dotrackpid"]
inputdir = data_param[case]["output_folders"]["pkl_out"][mcordata][indexp]
outputdir = data_param[case]["output_folders"]["pkl_skimmed"][mcordata][indexp]
maxfiles = data_config["skimming"][mcordata]["maxfiles"]
nmaxconvers = data_config["skimming"][mcordata]["nmaxconvers"]

listfilespath, listfilespathevt, listfilespathgen, \
listfilespathout, listfilespathoutevt, listfilespathoutgen = \
    list_create_dir(inputdir, outputdir, namefile_reco, namefile_evt, namefile_gen, \
                    namefile_reco_skim, namefile_evt_skim, namefile_gen_skim, maxfiles)


In [8]:
chunks, chunksout = createchunks(listfilespath, listfilespathout, nmaxconvers)
chunksevt, chunksoutevt = createchunks(listfilespathevt, listfilespathoutevt, nmaxconvers)

In [9]:
# for index, _ in enumerate(chunks):
#     print("Processing chunk number=", index)
#     skimall(chunks[index], chunksevt[index], chunksout[index],
#             skimming_sel, var_evt_match, param_case, presel_reco, sel_cent,
#             skimming2_dotrackpid)


In [10]:
import pandas as pd
import pickle
import multiprocessing as mp


def newskimmer(filein, skimming_sel):
    df = pickle.load(open(filein, "rb"))
    print("DONE")

In [11]:
def parallelizer(function, argument_list, maxperchunk = 2):
    chunks = [argument_list[x:x+maxperchunk]  for x in range(0, len(argument_list), maxperchunk)]
    for chunk in chunks:
        print(chunk)
        pool = mp.Pool(10)
        [pool.apply(function,args=chunk[i]) for i in range(len(chunk))] 
        pool.close()

In [12]:
argumentl = [("files/child_1/0033/AnalysisResultsRecoLctopK0s.pkl", skimming_sel), ("files/child_1/0033/AnalysisResultsRecoLctopK0s.pkl", skimming_sel), ("files/child_1/0033/AnalysisResultsRecoLctopK0s.pkl", skimming_sel), ("files/child_1/0033/AnalysisResultsRecoLctopK0s.pkl", skimming_sel)]

In [13]:
parallelizer(newskimmer, argumentl)

[('files/child_1/0033/AnalysisResultsRecoLctopK0s.pkl', 'pt_cand>4 & is_ev_rej==0'), ('files/child_1/0033/AnalysisResultsRecoLctopK0s.pkl', 'pt_cand>4 & is_ev_rej==0')]
DONE
DONE
[('files/child_1/0033/AnalysisResultsRecoLctopK0s.pkl', 'pt_cand>4 & is_ev_rej==0'), ('files/child_1/0033/AnalysisResultsRecoLctopK0s.pkl', 'pt_cand>4 & is_ev_rej==0')]
DONE
DONE


In [14]:
# def newskimmermap(filein, sel):
#     df = pickle.load(open(filein, "rb"))
#     print("DONE")
    
# def parallelizer_map(function, argument_list):
#     pool = mp.Pool(10)
#     pool.map(function, argument_list)
#     pool.close()
# argumentlmap = [("files/child_1/0033/AnalysisResultsRecoLctopK0s.pkl", "1"), ("files/child_1/0033/AnalysisResultsRecoLctopK0s.pkl", "2")]
# parallelizer_map(newskimmermap, argumentlmap)